### Homework week 9

In [1]:
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.xception import preprocess_input

import tensorflow.lite as tflite

In this homework, we'll deploy the dino or dragon model we trained in the previous homework.

Download the model from here:

In [3]:
!wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/dino-dragon-model/dino_dragon_10_0.899.h5

In [4]:
model = keras.models.load_model('dino_dragon_10_0.899.h5')

Now convert this model from Keras to TF-Lite format.

In [5]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

### Generate a tf-lite file storing model
with open('hw09-model.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\rizdi\AppData\Local\Temp\tmpcgedibl0\assets


INFO:tensorflow:Assets written to: C:\Users\rizdi\AppData\Local\Temp\tmpcgedibl0\assets


### Question 1

What's the size of the converted model?

#### Answer: 44.8 -> 43 Mb

In [2]:
interpreter = tflite.Interpreter(model_path='hw09-model.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

In [3]:
interpreter.get_output_details()

[{'name': 'StatefulPartitionedCall:0',
  'index': 13,
  'shape': array([1, 1]),
  'shape_signature': array([-1,  1]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [4]:
output_index

13

### Question 2

To be able to use this model, we need to know the index of the input and the index of the output.

What's the output index for this model?
#### Answer: 13

Now we need to turn the image into numpy array and pre-process it.

In [5]:
from io import BytesIO
from urllib import request
from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [13]:
!wget https://upload.wikimedia.org/wikipedia/commons/thumb/d/df/Smaug_par_David_Demaret.jpg/1280px-Smaug_par_David_Demaret.jpg

In [6]:
url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/d/df/Smaug_par_David_Demaret.jpg/1280px-Smaug_par_David_Demaret.jpg'

image_input = download_image(url)

In [25]:
image_preprocessed = prepare_image(image_input, (150, 150))

Based on the previous homework, what should be the target size for the image? 150 x 150

In [26]:
def preprocess_input(x):
    x /= 255
    # x -= 1.
    return x

x = np.array(image_preprocessed, dtype='float32')
X = np.array([x])

X = preprocess_input(X)

In [27]:
X[0][0][0]

array([0.5529412 , 0.31764707, 0.1764706 ], dtype=float32)

### Question 3

After the pre-processing, what's the value in the first pixel, the R channel?

#### Answer: 0.5529412

In [28]:
input_index

0

In [29]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)
preds

array([[0.82448506]], dtype=float32)

### Question 4

Now let's apply this model to this image. What's the output of the model?

#### Answer: 0.82448506 -> 0.82448614


### Prepare the lambda code

Now you need to copy all the code into a separate python file. You will need to use this file for the next two questions.

### Docker

For the next two questions, we'll use a Docker image that we already prepared. This is the Dockerfile that we used for creating the image:
```
FROM public.ecr.aws/lambda/python:3.9
COPY dino-vs-dragon-v2.tflite .
```

And pushed it to `svizor42/zoomcamp-dino-dragon-lambda:v2`

### Question 5

Download the base image svizor42/zoomcamp-dino-dragon-lambda:v2. You can easily make it by using docker pull command.

So what's the size of this base image?

#### Answer: 639 Mb


### Question 6

Now let's extend this docker image, install all the required libraries and add the code for lambda.

You don't need to include the model in the image. It's already included. The name of the file with the model is dino-vs-dragon-v2.tflite and it's in the current workdir in the image (see the Dockerfile above for the reference).

Now run the container locally.

Score this image: https://upload.wikimedia.org/wikipedia/en/e/e9/GodzillaEncounterModel.jpg

What's the output from the model?

#### Answer: {'prediction': 0.31950676441192627} -> 0.32


### Further work

### Publishing it to AWS

Now you can deploy your model to AWS!

    Publish your image to ECR
    Create a lambda function in AWS, use the ECR image
    Give it more RAM and increase the timeout
    Test it
    Expose the lambda function using API Gateway
